In [ ]:
##  THIS CODE WILL NOT WORK ON ITS OWN IN THIS REPO - Contact Huriye for the full code

%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle



# Get the list of recordings
info = mfun.analysis()
# display the detected session
print( "Total Session: " +  str(info.recordingList .shape[0]))
#info.recordingList.head()

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')


Takes <30s. However, it might take >1 mins or so if network is busy...
Computer: Huriye Windows


c:\Users\Huriye\Documents\code\decision-making-ev\main_funcs.py:141: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-05-18_OFZ008_1\' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.recordingList.loc[ind,'analysispathname'] = analysispathname +'\\'


Total Session: 215
Matlab engine is set correctly.


In [ ]:
# # to check the paq file details - change filenamePAQ
# import paq2py as paq_reader
# import utils_funcs as utils

# filenamePAQ =  'Y:\\OFZ011\\2023-07-07\\TwoP\\2023-07-07_OFZ011_paq_001.paq'
# savepathname = info.recordingList()
# paq_data = paq_reader.paq_read( file_path=filenamePAQ, plot=True, save_path=savepathname) 
# frame_clock = utils.paq_data (paqData, 'reward', threshold_ttl=True, plot=False)

In [9]:
# Check CSV files 
# checkOnly needs to be False to create neww files with MATLAB GENERIC CODE: GetBehavData.mat
checkOnly = True # Make false when there is more behaviour session
alignSubtract = True
#ind = 49
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
#if ind ==49:
    filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
    e_filenameCSV = [f for f in glob.glob(filenameCSV)]
    if len(e_filenameCSV)==1:
        info.recordingList.loc[ind,'eventTimesExtracted']=1
        info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
    else:
        if checkOnly:
            info.recordingList.loc[ind,'eventTimesExtracted']=0
            info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
        else:
            try:
                filenameTimeline = [f for f in glob.glob(info.recordingList.filepathname[ind]+ '\\' + info.recordingList.sessionName[ind] + '_Timeline.mat')]
                if  (len(filenameTimeline)>0):
                    sessionProfile ='Value2AFC'
                else:
                    sessionProfile ='Value2AFC_noTimeline'
                
                # Get behaviour trial data from Block.mat file
                print(' Extracting time events--> Profile: ' + sessionProfile  +'  Session: ' + info.recordingList.sessionName[ind])
                data = eng.getBehavData(info.recordingList.sessionName[ind],sessionProfile)

                # Apply correction based on the weights to match the eventTimes
                if (alignSubtract) & (len(filenameTimeline)>0) & (info.recordingList.twoP[ind]==True): 
                    print(' Aligning time events: ' + info.recordingList.sessionName[ind])
                    # Get weights to convert from probe to behavioural timebase
                    twoPpath = info.recordingList.path[ind] + '\\TwoP'
                    sessionName = info.recordingList.sessionName[ind]
                    figsavepath = info.recordingList.analysispathname[ind]
                    dataCorrected, variance = eng.applySubtractionCorrection (data, twoPpath ,sessionName, True, figsavepath, nargout=2)
                    info.recordingList.loc[ind,'variance'] = variance

                    # Apply correction to the signal
                    data = dataCorrected
                    
                # Save the file
                eng.writetable(data, filenameCSV, nargout=0)
                info.recordingList.loc[ind,'eventTimesExtracted']=1
            except:
                print(str(ind) + ' - FAILED: Extracting time events: ' + info.recordingList.sessionName[ind])
                info.recordingList.loc[ind,'eventTimesExtracted']=0

# display the output
print( "Behaviour trial data extraction completed: " + 
      str(info.recordingList.eventTimesExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

Behaviour trial data extraction completed: 141.0/215


In [10]:
# DONOT RUN IF NOT NEEDED: Check Suite2p, calculate dff, seperate it for each sessions

animalList = np.unique(info.recordingList.animalID)
for animal in animalList:
    filtered_sessionOneAnimal =  info.recordingList[(info.recordingList['animalID'] == animal) &
                                                     (info.recordingList['twoP'] == True) &
                                                     (info.recordingList['ROI'] >0)]
    roiList = np.unique(filtered_sessionOneAnimal.ROI)
    for roi in roiList:
        print('Animal: ' + str(animal) + ' ROI:' + str(roi) )
        # Get the frame numbers for each session (analysisPath)
        s2p_path = os.path.join(info.suite2pOutputPath, animal,'suite2p_ROI-'+ format(roi, '03d'),'plane0')
        if os.path.exists(s2p_path):
            print('Extracting...')
            ops = np.load(s2p_path+'\\ops.npy', allow_pickle=True)
            ops = ops.item()
            FrameNums = ops['frames_per_file']
            filelist = ops['filelist']
            isCell = np.load(s2p_path+'\\iscell.npy', allow_pickle=True)

            # Load the suite2p
            flu_raw_subtracted, spks, stat = utils.s2p_loader(s2p_path)
            fluAll = utils.dfof2(flu_raw_subtracted)

            # Cut each session & save it in the analysis-session folder
            t = 0
            for idx, ses_len in enumerate(FrameNums):
                start_frame = t
                end_frame   = t + ses_len
                t = t + ses_len
                flu =fluAll[:,start_frame:end_frame]

                imaging_data = { "n_frames":ses_len,
                                 "flu": flu,
                                 "spks": spks,
                                 "stat": stat,
                                }
                
                # save flu for each session
                ses_ind = filelist[idx]
                if  len(ses_ind.split('\\'))>2:
                    recordingDate = ses_ind.split('\\')[2]  # Assuming the animal ID is the part before the first underscore
                    animal_id     = ses_ind.split('\\')[1]
                else:
                    recordingDate = ses_ind.split('/')[2]  # Assuming the animal ID is the part before the first underscore
                    animal_id     = ses_ind.split('/')[1]

                recording_id  = int(ses_ind.split('-')[-1][:3])
                recordingList_index = np.where((info.recordingList['animalID'] == animal) & (info.recordingList['recordingDate'] == recordingDate))# & (info.recordingList['recordingID'] == str(recording_id)))
                if len(recordingList_index[0])>1:
                    print('working')
                    recordingList_index = np.where((info.recordingList['animalID'] == animal) & (info.recordingList['recordingDate'] == recordingDate) & (info.recordingList['recordingID'] == str(recording_id)))
                    
                'Y:/OFZ011/2023-06-05/TwoP/2023-06-05_OFZ011_t-001\\2023-06-05_OFZ011_t-001_Cycle00001_Ch2.tif',
                filenameINFO = info.recordingList.analysispathname[recordingList_index[0][0]] + 'imaging-data.pkl'
                print(filenameINFO + ' --> # of cells:' + str(flu.shape[0]) + '# of frames:' + str(flu.shape[1]))
                with open(filenameINFO, 'wb') as f:
                    pickle.dump(imaging_data, f)

print('Seperating each session from truncated trace is completed.')            
        

Animal: OFZ008 ROI:2
Animal: OFZ008 ROI:3
Extracting...
loading 120 traces labelled as cells
subtracting neuropil with a coefficient of 0.7
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-14_OFZ008_1\imaging-data.pkl --> # of cells:120# of frames:85000
working
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-16_OFZ008_1\imaging-data.pkl --> # of cells:120# of frames:41824
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-17_OFZ008_1\imaging-data.pkl --> # of cells:120# of frames:125470
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-19_OFZ008_1\imaging-data.pkl --> # of cells:120# of frames:105000
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-20_OFZ008_1\imaging-data.pkl --> # of cells:120# of frames:105000
Animal: OFZ008 ROI:4
Extracting...
loading 145 traces labelled as cells
subtracting neuropil with a coefficient of 0.7
C:\Users\Huriye\Documents\code\decision-making-ev\analysis\2023-07-24_

In [11]:
# Check Suite2p files
# Run: suite2p_run.py to create files in suite2p enviroment. 
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    filenameDFF = info.recordingList.analysispathname[ind] + 'imaging-data.pkl'
    e_filenameDFF = [f for f in glob.glob(filenameDFF)]
    if len(e_filenameDFF)>0:
        info.recordingList.loc[ind,'imagingDataExtracted']=1
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF
    else:
        info.recordingList.loc[ind,'imagingDataExtracted']=0
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF

# display the output
print( "Imaging data extraction completed: " + 
      str(info.recordingList.imagingDataExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

Imaging data extraction completed: 42.0/215


In [12]:
# Save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

# Save table as CSV
recordingList = info.recordingList
recordingList.to_csv( info.analysisPath +'\\recordingList.csv', index=False)

All should be done!!
